In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from langdetect import detect
import time
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
import numpy as np
import json

import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import pickle

import matplotlib.pyplot as plt
import joblib
import random
import nltk, re, time
from nltk.corpus import stopwords
import string
from collections import defaultdict
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical
from collections import namedtuple

from contractions import get_contractions
import operator
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import tensor_array_ops
print('TensorFlow Version: {}'.format(tf.__version__))

alreadyPickled = False

TensorFlow Version: 1.4.0


## Variable Initialization

In [3]:
embedding_dim = 300
num_layers = 2
num_classes = 5
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.01
keep_probability = 0.75
max_sequence_length = 1000


In [14]:
def clean_text(text, remove_stopwords = True):
    text = text.lower()
    text = text.translate((None, string.punctuation))
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

In [10]:
restId = []
for line in open('./dataset/business.json', 'r'):
    data = json.loads(line)
    if 'Restaurants' in data['categories'] or 'Food' in data['categories']:
        restId.append(data['business_id'])
print("Done getting Business IDs")
print("There are %d restaurants" % (len(restId)))

Done getting Business IDs
There are 65028 restaurants


In [22]:
contractions = get_contractions()

revs_list = [[]]
stars_list = [[]]
num = 100000 # Number of review read
k = 0 # Count
first = True
for line in open('./dataset/review.json', 'r'):
    if first:
        first = False
        continue
    if k >= num:
        break
    data = json.loads(line)
    text = data['text']
    star = data['stars']
    ID = data['business_id']
    # Check language
    if detect(text) != 'en':
        continue  
    # Check whether it is a restaurant
    if ID not in restId:
        continue
    revs_list.append(clean_text(text))
    stars_list.append(star)
    k += 1
    # Notify for every 500 reviews
    if len(revs_list) % 1000 == 0:
        print(len(revs_list))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


In [23]:
print(revs_list[1])
print(len(revs_list), len(stars_list))

place horrible excited try since got gift card birthday went ordered whole meal except gift card system unacceptable would helpful would known prior
100001 100001


In [24]:
zipped_revs = zip(revs_list, stars_list)
print("Shape of Ziiped Revs is [%d, %d]" % (len(zipped_revs), len(zipped_revs[0])))
categories = ['text', 'stars']
df_reviews_processing = pd.DataFrame(zipped_revs, columns=categories)

Shape of Ziiped Revs is [100001, 2]


In [21]:
df_reviews_processing.shape
df_reviews_processing.head()
df_reviews_processing[['stars']] = df_reviews_processing[['stars']].apply(pd.to_numeric)

df_reviews_processing = df_reviews_processing[np.isfinite(df_reviews_processing['stars'])]

print("Before Dropping Nones: Shape %d,%d" % (df_reviews_processing.shape[0], df_reviews_processing.shape[1]))
df_reviews = df_reviews_processing[np.isfinite(df_reviews_processing['stars'])]
print("After Dropping Nones: Shape %d,%d" % (df_reviews_processing.shape[0], df_reviews_processing.shape[1]))

df_reviews_processing.to_csv("reviews_df_processed100k.csv", encoding='utf-8')


# Data Loading
If the data is already pickled, then can skip embedding and data processing

In [39]:
def picklefiles(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def loadfiles(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

## Embeddings
Using ConceptNet Numberbatch instead of GLoVE (supposedly outperforms GLoVE embeddings)  
  
  
On top of the embeddings, we also keep track of commonly used words in the reviews that Embeddings don't cover. This way we could have higher test accuracy when words we come across words like these. This is specified by a threshold value. Currently, threshold is set to 20 occuraces.  
  
  
We also process the reviews a bit more, sorting them into comparable lengths. This way, there is less padding necessary and (possibly) faster computation time when training/testing

In [40]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [83]:
word_counts = {}
count_words(word_counts, df_reviews.text)            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 15259


In [46]:
embed_path='./embeddings/numberbatch-en-17.02.txt'
def load_embeddings(path='./embeddings/numberbatch-en-17.02.txt'):
    embeddings_index = {}
    with open(path) as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embedding
    return embeddings_index

embeddings_index = load_embeddings(embed_path)

# Balance the Dataset
Want to balance the dataset, such that we have an equal number of reviews for each different category.  
For example, if our distribution of reviews is [200,500,100,300,400], for [1,2,3,4,5] stars, respectively, then we will only take 100 of each review

In [55]:
def balance_dataframe(df, category=['stars'], downsample_by=0.9):
    """
    :param df: pandas.DataFrame
    :param categorical_columns: iterable of categorical columns names contained in {df}
    :return: balanced pandas.DataFrame
    """
    if category is None or not all([col in df.columns for col in category]):
        raise ValueError('Please provide one or more columns containing categorical variables')

    lowest_count = df.groupby(category).apply(lambda x: x.shape[0]).min()
    # Remove 10 percent of the reviews
    lowest_count = int(lowest_count * downsample_by)
    
    df = df.groupby(category).apply( 
        lambda x: x.sample(lowest_count)).drop(category, axis=1).reset_index().set_index('level_1')

    df.sort_index(inplace=True)

    return df

In [74]:
# df_reviews = pd.read_csv('reviewsText.csv')
df_reviews = pd.read_csv("reviews_df_processed.csv")
df_reviews['len'] = df_reviews.text.str.len()
df_reviews = df_reviews[df_reviews['len'].between(10, 4000)]
df_reviews[['stars']] = df_reviews[['stars']].apply(pd.to_numeric)

print("(pre) 1 star ratings: %d" % (len(df_reviews[df_reviews.stars == 1])))
print("(pre) 2 star ratings: %d" % (len(df_reviews[df_reviews.stars == 2])))
print("(pre) 3 star ratings: %d" % (len(df_reviews[df_reviews.stars == 3])))
print("(pre) 4 star ratings: %d" % (len(df_reviews[df_reviews.stars == 4])))
print("(pre) 5 star ratings: %d" % (len(df_reviews[df_reviews.stars == 5])))

# Set downsample_by to 0.9, since not much training data to begin with
df_balanced = balance_dataframe(df_reviews, 
                                category=['stars'], 
                                downsample_by=0.9)

df_balanced.to_csv('balanced_reviews.csv', encoding='utf-8')
print("(post) 1 star ratings: %d" % (len(df_balanced[df_balanced.stars == 1])))
print("(post) 2 star ratings: %d" % (len(df_balanced[df_balanced.stars == 2])))
print("(post) 3 star ratings: %d" % (len(df_balanced[df_balanced.stars == 3])))
print("(post) 4 star ratings: %d" % (len(df_balanced[df_balanced.stars == 4])))
print("(post) 5 star ratings: %d" % (len(df_balanced[df_balanced.stars == 5])))




(pre) 1 star ratings: 469
(pre) 2 star ratings: 545
(pre) 3 star ratings: 759
(pre) 4 star ratings: 1353
(pre) 5 star ratings: 1874
(post) 1 star ratings: 422
(post) 2 star ratings: 422
(post) 3 star ratings: 422
(post) 4 star ratings: 422
(post) 5 star ratings: 422


In [84]:
embedding_dim = 300
MAX_NB_WORDS = 20000

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)

joblib.dump(tokenizer, 'tokenizer.p')

WORD_INDEX_SORTED = sorted(tokenizer.word_index.items(), key=operator.itemgetter(1))
print("number of words is %d" % (len(WORD_INDEX_SORTED)))

NB_WORDS = min(len(WORD_INDEX_SORTED), MAX_NB_WORDS)

word_embedding_matrix = np.zeros((NB_WORDS, embedding_dim),
                                 dtype=np.float32)

for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i-1] = embedding_vector
print("embedding matrix size: is [%d,%d]" % (word_embedding_matrix.shape[0], 
                                            word_embedding_matrix.shape[1]))

number of words is 23


IndexError: index 23 is out of bounds for axis 0 with size 23

In [75]:

seqs = tokenizer.texts_to_sequences(df_balanced.text.values)

padReviews = pad_sequences(seqs, maxlen=MAX_SEQUENCE_LENGTH)

reviewsLength = MAX_SEQUENCE_LENGTH

ratings = df_balanced.stars.values.astype(int)
ratings_cat = to_categorical(ratings)

X_train, X_test, y_train, y_test = train_test_split(padReviews, ratings_cat, test_size=0.2, random_state=9)
with pd.HDFStore('x_y_test_train.h5') as store:
    store['X_train'] = pd.DataFrame(X_train)
    store['X_test'] = pd.DataFrame(X_test)
    store['y_train'] = pd.DataFrame(y_train)
    store['y_test'] = pd.DataFrame(y_test)

assert padReviews.shape[0] == ratings.shape[0] 

NameError: name 'NB_WORDS' is not defined

In [ ]:
embedding_dim = 300
MAX_NB_WORDS = 20000

NB_WORDS = min(len(WORD_INDEX_SORTED), MAX_NB_WORDS)

word_embedding_matrix = np.zeros((NB_WORDS, embedding_dim),
                                 dtype=np.float32)

for word, i in tokenizer.word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

In [ ]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_texts = create_lengths(int_texts)
print("Texts:")
print(lengths_texts.describe())

In [ ]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 80))
print(np.percentile(lengths_texts.counts, 85))
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

In [ ]:
if alreadyPickled == False:
    picklefiles("./data/good_pickles/clean_reviews.p",clean_reviews)
    picklefiles("./data/good_pickles/sorted_reviews.p",sorted_reviews)
    picklefiles("./data/good_pickles/word_embedding_matrix.p",word_embedding_matrix)
    
    df_rev_balanced = pd.read_csv('balanced_reviews.csv')
    tokenizer = joblib.load('tokenizer.pickle')
    with pd.HDFStore('x_y_test_train.h5') as h:
        X_train = h['X_train'].values
        X_test = h['X_test'].values
        y_train = h['y_train'].values
        y_test = h['y_test'].values

# Gettin Batches
Gets batches as well as pads them to have similar length

In [ ]:
def get_batches(ratings, reviews, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(reviews)//batch_size):
        start_i = batch_i * batch_size
        rating_batch = ratings[start_i:start_i + batch_size, :]
        reviews_batch = reviews[start_i:start_i + batch_size, :]

        yield rating_batch, reviews_batch

# Building the Graph

In [ ]:
def model_inputs():

    # Should be [batch_size x review length]
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    # Should be [batch_size x num_classes]
    labels = tf.placeholder(tf.int32, [None, None], name='labels')
    
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, labels, lr, keep_prob

In [ ]:
input_data, labels, lr, keep_prob = model_inputs()

vocab_size = len(WORD_INDEX_SORTED)+1

# def prediction_model(input_data, target_data, keep_prob, num_classes 
#                      vocab_size, rnn_size, num_layers, batch_size):
    
W2 = tf.Variable(np.random.rand(rnn_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1, num_classes)), dtype=tf.float32)

embeddings = word_embedding_matrix
embs = tf.nn.embedding_lookup(embeddings, input_data)

stacked_rnn = []
for layer in range(num_layers):
    cell_fw = tf.contrib.rnn.GRUCell(rnn_size,
                                     initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw,
                                            input_keep_prob = keep_prob)
    stacked_rnn.append(cell_fw)
    
multilayer_cell = tf.contrib.rnn.MultiRNNCell(stacked_rnn, state_is_tuple=True)

output, states_series = tf.nn.dynamic_rnn(cell_fw,
                                   rnn_inputs,
                                   dtype=tf.float32)
# states_series_reshaped = tf.reshape(states_series, [-1, rnn_size])

# logits = tf.matmul(states_series_reshaped, W2) + b2 #Broadcasted addition
# labels_logits = tf.reshape(labels, [-1])


# logits_series = tf.unstack(tf.reshape(logits, [batch_size, num_classes]), axis=1)
# predictions_series = [tf.nn.softmax(logit) for logit in logits_series]



In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    
    # Create tensors for the training logits and inference logits
#     training_logits = tf.identity(training_logits[0].rnn_output, 'logits')
#     inference_logits = tf.identity(inference_logits[0].sample_id, name='predictions')
    
    
    # Create the weights for sequence_loss, the sould be all True across since each batch is padded
    
    with tf.name_scope("optimization"):
        # Loss function
        
        cost = tf.losses.softmax_cross_entropy(labels,
                                               input_data)
        total_loss = tf.reduce_mean(cost)

        # Optimizer
        optimizer = tf.train.RMSOptimizer(learning_rate).minimize(total_loss)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
        
print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)

# Training the Model


In [ ]:
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(X_train)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model


In [ ]:
checkpoint = "./saves/best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (trainX_batch, trainY_batch) in enumerate(get_batches(X_train, y_train, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [optimizer, total_loss],
                {input_data: trainX_batch,
                 targets: trainY_batch,
                 lr: learning_rate,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(X_train) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break